In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls

drive  MNIST  sample_data


請注意這是cpu版!!!!!!

In [3]:
import torch
from torchvision import datasets, transforms

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import matplotlib.pyplot as plt

In [4]:
class Netwrok(nn.Module):
  def __init__(self):
    super(Netwrok, self).__init__()
    # imge 28x28=784
    self.input_layer = nn.Linear(784, 64)
    self.hidden_layer1 = nn.Linear(64, 64)
    self.hidden_layer2 = nn.Linear(64, 64)
    self.output = nn.Linear(64, 10)
  def forward(self,data):
    data = F.relu(self.input_layer(data))
    data = F.relu(self.hidden_layer1(data))
    data = F.relu(self.hidden_layer2(data))
    data = self.output(data)
    return F.log_softmax(data,dim=1)

In [5]:
training = datasets.MNIST("",train=True,download=True,transform=transforms.Compose([transforms.ToTensor()]))
testing = datasets.MNIST("",train=False,download=True,transform=transforms.Compose([transforms.ToTensor()]))

In [6]:
train_set = torch.utils.data.DataLoader(training,batch_size=10,shuffle=True)
test_set = torch.utils.data.DataLoader(training,batch_size=10,shuffle=True)

In [7]:
netwrok=Netwrok()

In [8]:
learn_rate = optim.Adam(netwrok.parameters(),lr=0.001)
epochs = 10

In [9]:
for i in range(epochs):
  for data in train_set:
    image,output = data
    netwrok.zero_grad()
    result = netwrok(image.view(-1,784))
    loss = F.nll_loss(result,output)
    loss.backward()
    learn_rate.step()
  print(loss)

tensor(0.0396, grad_fn=<NllLossBackward0>)
tensor(0.0008, grad_fn=<NllLossBackward0>)
tensor(0.0031, grad_fn=<NllLossBackward0>)
tensor(0.0037, grad_fn=<NllLossBackward0>)
tensor(0.0015, grad_fn=<NllLossBackward0>)
tensor(0.0009, grad_fn=<NllLossBackward0>)
tensor(0.0005, grad_fn=<NllLossBackward0>)
tensor(0.0005, grad_fn=<NllLossBackward0>)
tensor(0.0006, grad_fn=<NllLossBackward0>)
tensor(0.0169, grad_fn=<NllLossBackward0>)


In [10]:
netwrok.eval()

corrcoef = 0
total = 0
with torch.no_grad():
  for data in test_set:
    image,output = data
    result = netwrok(image.view(-1,784))
    for index,tensor_value in enumerate(result):
      total += 1
      if torch.argmax(tensor_value) == output[index]:
        corrcoef += 1
accuracy = corrcoef/total
print(f"Accuracy:{accuracy}")

Accuracy:0.9872833333333333


請注意這是GPU版!!!!!!

In [11]:
import torch
from torchvision import datasets, transforms

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import matplotlib.pyplot as plt
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print('GPU')


GPU


In [12]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    # imge 28x28=784
    self.input_layer = nn.Linear(784, 64)
    self.hidden_layer1 = nn.Linear(64, 64)
    self.hidden_layer2 = nn.Linear(64, 64)
    self.output = nn.Linear(64, 10)
  def forward(self,data):
    data = F.relu(self.input_layer(data))
    data = F.relu(self.hidden_layer1(data))
    data = F.relu(self.hidden_layer2(data))
    data = self.output(data)
    return F.log_softmax(data,dim=1)

In [13]:
training = datasets.MNIST("",train=True,download=True,transform=transforms.Compose([transforms.ToTensor()]))
testing = datasets.MNIST("",train=False,download=True,transform=transforms.Compose([transforms.ToTensor()]))

In [14]:
train_set = torch.utils.data.DataLoader(training,batch_size=10,shuffle=True)
test_set = torch.utils.data.DataLoader(training,batch_size=10,shuffle=True)

In [15]:
network=Network().to(device)

In [16]:
learn_rate = optim.Adam(netwrok.parameters(),lr=0.001)
epochs = 10

In [19]:
for i in range(epochs):
  for data in train_set:
    image,output = data
    image = image.to(device)
    output = output.to(device)
    netwrok.zero_grad()
    result = network(image.view(-1,784))
    loss = F.nll_loss(result,output)
    loss.backward()
    learn_rate.step()
  print(loss)

tensor(2.2932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2871, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3193, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3298, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3322, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2869, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2663, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3585, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2840, device='cuda:0', grad_fn=<NllLossBackward0>)


In [20]:
network.eval()

corrcoef = 0
total = 0
with torch.no_grad():
  for data in test_set:
    image,output = data
    image = image.to(device)
    output = output.to(device)
    result = network(image.view(-1,784))
    for index,tensor_value in enumerate(result):
      total += 1
      if torch.argmax(tensor_value) == output[index]:
        corrcoef += 1
accuracy = corrcoef / total
print(f"Accuracy:{accuracy}")

Accuracy:0.09035
